In [91]:
import ast

In [33]:
cmd = """
def myfunc(a):
    print('hello', a)
    
def otherfunc(b):
    return b + 10
    
myfunc('there')
print(otherfunc(10))
"""

In [34]:
tree = ast.parse(cmd)

In [35]:
tree

In [36]:
exec(compile(tree, filename="<ast>", mode="exec"))

hello there
20


In [37]:
tree._fields

('body',)

In [38]:
tree._attributes

()

In [39]:
tree

In [40]:
class FuncLister(ast.NodeVisitor):
    def visit_FunctionDef(self, node):
        print(node.name)
        self.generic_visit(node)

FuncLister().visit(tree)

myfunc
otherfunc


In [46]:
for node in ast.walk(tree):
    print(ast.dump(node) + "\n\n")

Module(body=[FunctionDef(name='myfunc', args=arguments(args=[arg(arg='a', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Str(s='hello'), Name(id='a', ctx=Load())], keywords=[]))], decorator_list=[], returns=None), FunctionDef(name='otherfunc', args=arguments(args=[arg(arg='b', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Name(id='b', ctx=Load()), op=Add(), right=Num(n=10)))], decorator_list=[], returns=None), Expr(value=Call(func=Name(id='myfunc', ctx=Load()), args=[Str(s='there')], keywords=[])), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Call(func=Name(id='otherfunc', ctx=Load()), args=[Num(n=10)], keywords=[])], keywords=[]))])


FunctionDef(name='myfunc', args=arguments(args=[arg(arg='a', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value

In [48]:
x = None
for node in ast.walk(tree):
    if isinstance(node, ast.BinOp):
        print(ast.dump(node) + "\n\n")
        x = node

BinOp(left=Name(id='b', ctx=Load()), op=Add(), right=Num(n=10))




In [52]:
x._attributes

('lineno', 'col_offset')

In [57]:
help(x)

Help on BinOp in module _ast object:

class BinOp(expr)
 |  Method resolution order:
 |      BinOp
 |      expr
 |      AST
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  _fields = ('left', 'op', 'right')
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from expr:
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes inherited from expr:
 |  
 |  _attributes = ('lineno', 'col_offset')
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from AST:
 |  
 |  __delattr__(self, name, /)
 |      Implement delattr(self, name).
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.


In [58]:
x.__dict__

{'left': <_ast.Name at 0x7f0577fb19e8>,
 'op': <_ast.Add at 0x7f0593478780>,
 'right': <_ast.Num at 0x7f0577fb1a20>,
 'lineno': 6,
 'col_offset': 11}

In [59]:
x.lineno

6

In [60]:
x.col_offset

11

In [65]:
x.left.id

'b'

In [66]:
x.op

In [67]:
isinstance(x.op, ast.Add)

True

In [72]:
n1 = ast.BinOp(left=ast.Name(id='b', ctx=ast.Load()),
               op=ast.Add(), right=ast.Num(n=10))

In [73]:
n1

In [74]:
x

In [82]:
x.__dict__

{'left': <_ast.Name at 0x7f0577fb19e8>,
 'op': <_ast.Add at 0x7f0593478780>,
 'right': <_ast.Num at 0x7f0577fb1a20>,
 'lineno': 6,
 'col_offset': 11}

In [83]:
class RewriteAddSub(ast.NodeTransformer):

    def visit_BinOp(self, node):
        return ast.copy_location(
            ast.BinOp(left=node.left,
                      op=ast.Sub(),
                      right=node.right),
            node)


In [84]:
tree2 = RewriteAddSub().visit(tree)

In [86]:
tree2

In [88]:
ast.dump(tree2)

"Module(body=[FunctionDef(name='myfunc', args=arguments(args=[arg(arg='a', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Str(s='hello'), Name(id='a', ctx=Load())], keywords=[]))], decorator_list=[], returns=None), FunctionDef(name='otherfunc', args=arguments(args=[arg(arg='b', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Name(id='b', ctx=Load()), op=Sub(), right=Num(n=10)))], decorator_list=[], returns=None), Expr(value=Call(func=Name(id='myfunc', ctx=Load()), args=[Str(s='there')], keywords=[])), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Call(func=Name(id='otherfunc', ctx=Load()), args=[Num(n=10)], keywords=[])], keywords=[]))])"

In [89]:
exec(compile(tree2, filename="<ast>", mode="exec"))

hello there
0
